# Quantum Dynamics Lie Suzuki Trotter

We use a the split operator method to simulate a one dimensional wavepacket going through a gaussian potential barrier(to avoid numerical oscillations).
Our hamiltonian has the standard time independent form(natural units):
$$H = \frac{p^2}{2} + V$$
We use as starting wavefunction a gaussian wavepacket traveling to the right with momentum $p_0$. 


In [ ]:
%reset -f
import numpy as np
import scipy.fftpack as f
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib notebook 

#initiate x vector
N_x = 2 ** 11 #power of two for FFT efficiency
x_min = 0
x_max = 7
dx = (x_max - x_min) / N_x
x = np.arange(x_min,x_max, dx)
p = (np.arange(N_x)) * (2 * np.pi) / (dx * N_x)
p[N_x // 2 : N_x] -= p[N_x - 1] + (2 * np.pi / (dx * N_x)) #Allow for negative momenta (initially ignored by DFT)


#initiate gaussian wavepacket centered at x0
x0 = 3.5
sigma =0.05
p0 = 100
psi0 = 1/np.sqrt(sigma*np.sqrt(2*np.pi))*np.exp(-(x-x0)**2/(4*sigma**2) + 1j*p0*(x))

#Gaussian barrier
V0 = 0.9 * p0 ** 2 / 2
V = V0 *np.exp(-(x-4)**2/0.1**2) +0j


#create imaginary optical boundary for damping the wave when it has reached the boundary(to avoid periodical bc)
dx_opt = 0.25 #width of boundary
U0 = 2*V0
x_opt = x_max - dx_opt 
opt_N = (int)(N_x * ((x_max - x_min - dx_opt) / (x_max - x_min)))
V[opt_N:-1] = -1j * U0 * (x[opt_N:-1] - x_opt) / dx_opt
V[0:N_x-opt_N] = 1j * U0 * (x[0:N_x-opt_N] - dx_opt) / dx_opt


N_t = 60000
dt = 1/2.5e6
psi_t = np.zeros((N_t,N_x)) +0.0j
psi_t[0,:] = psi0
V_matrix = np.exp(-1j * dt * V)
T_matrix = np.exp(-1j * dt * p ** 2 / 2)
for i in range(1,N_t):
    psi_t[i,:] = np.fft.ifft(T_matrix * np.fft.fft( V_matrix * psi_t[i-1,:]))


In [ ]:
plt.plot(x, abs(psi_t) ** 2 * V0)
plt.plot(x,V.real)
plt.plot(x,V.imag)
plt.title('Initial situation at t=0')
plt.xlabel('position')
plt.ylabel('Energy / Real part of Wavefunction * V0')

In [ ]:
fig, ax = plt.subplots()
speed_factor = 100
line, = ax.plot(x, abs(psi0) ** 2)
ax2 = ax.twinx()
ax2.plot(x,V.real,'r:')
ax2.plot(x,V.imag,'c:')
ax.set_ylim((-max(abs(psi0) ** 2), max(abs(psi0) ** 2)))
ax2.set_ylim((-max(abs(V.real)),max(abs(V.real))))

def animationFunc(i):
    #line.set_ydata(psi_t.real[i,:])  # update the data
    line.set_ydata(abs(psi_t[i * speed_factor,:]) **2)
    return line,

#Init only required for blitting to give a clean slate.
def init():
    line.set_ydata(np.ma.array(x, mask=True))
    return line,

ani = animation.FuncAnimation(fig, animationFunc, init_func=init, frames = (int)(N_t //speed_factor),
    interval=min(5 * speed_factor, 50), blit = True, repeat = False)
#display_animation(ani)

Now let's show the momemtum distribution over time

In [ ]:
fig, ax = plt.subplots()
speed_factor = 150
line, = ax.plot(p, abs(np.fft.fft(psi0)) ** 2)
ax.set_xlabel('Momentum')
ax.set_ylabel('FFT(psi)')
def animationFunc(i):
    #line.set_ydata(psi_t.real[i,:])  # update the data
    line.set_ydata(abs((np.fft.fft(psi_t[i * speed_factor,:]))) **2)
    return line,

#Init only required for blitting to give a clean slate.
def init():
    line.set_ydata(np.ma.array(x, mask=True))
    return line,

ani = animation.FuncAnimation(fig, animationFunc, init_func=init, frames = (int)(N_t //speed_factor),
    interval=50, blit = True, repeat = False)
#display_animation(ani)

In [ ]:
plt.figure()
plt.plot(p)

In [ ]:
%matplotlib notebook
plt.plot(p, abs(np.fft.fft(psi0)) ** 2)
plt.show()

In [ ]:
%reset -f
import numpy as np
import scipy.fftpack as f
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib notebook 

#initiate x vector
N_x = 2 ** 11 #power of two for FFT efficiency
x_min = -10
x_max = 10
dx = (x_max - x_min) / N_x
x = np.arange(x_min,x_max, dx)
p = (np.arange(N_x)) * (2 * np.pi) / (dx * N_x)
p[N_x // 2 : -1] = p[N_x // 2 : -1] - p[N_x-1] - (2 * np.pi / (dx * N_x))

#create potential well around -a,a
a = 2
V0 = 2000
V = np.zeros(N_x) +0j
left_N = (int)(N_x * (-(x_min + a) / (x_max - x_min)))
right_N = (int)(left_N + N_x * 2 * a / (x_max - x_min))
V[left_N : right_N] =  V0

#Gaussian well
#V = 900*np.exp(-(x-3)**2/0.3**2)

#create imaginary optical boundary for damping the wave when it has reached the boundary
dx_opt = 10 #width of boundary
U0 = 1
x_opt = x_max - dx_opt 
opt_N = (int)(N_x * ((x_max - x_min - dx_opt) / (x_max - x_min)))
#V[opt_N:-1] = -1j * U0 * (x[opt_N:-1] - x_opt) / dx_opt

#initiate gaussian wavepacket centered at x0
x0 = -3
sigma =0.05

p0 = 50
psi0 = 1/np.sqrt(sigma*np.sqrt(2*np.pi))*np.exp(-(x-x0)**2/(4*sigma**2) + 1j*p0*(x))


N_t = 1200
dt = 1/10000
psi_t = np.zeros((N_t,N_x)) +0.0j
psi_t[0,:] = psi0
V_matrix = np.exp(-1j * dt * V)
T_matrix = np.exp(-1j * dt * p ** 2 / 2)
for i in range(1,N_t):
    psi_t[i,:] = np.fft.ifft(T_matrix * np.fft.fft( V_matrix * psi_t[i-1,:]))


In [ ]:
print(V_matrix)